### 3.5 출력층 설계하기

신경망은 분류와 회귀 모두에 이용할 수 있다.
다만 어떤 문제냐에 따라 출력층에서 사용하는 활성화 함수가 달라진다.
회귀에는 항등 함수를, 분류에는 소프트맥스 함수를 사용한다.

#### 3.5.1 항등 함수와 소프트맥스 함수 구현하기

항등 함수(identity function)는 입력을 그대로 출력한다.
반면 분류에서 사용하는 소프트맥스 함수(softmax function)의 식은 e^x를 뜻하는 지수함수, 출력층의 뉴런 수 n, 그중 k번째 출력을 뜻하는 yk의 조합으로 나타낸다. 분자는 입력 신호의 an의 지수 함수, 분모는 모든 입력 신호의 지수 함수의 합으로 구성된다.

이전 층에서 나온 a를 양수로 만들고, 큰 값일수록 강조하기 위해 지수함수를 씌운다.
그리고 이를 각 노드별 지수함수를 씌운 값들의 총합으로 나누어 확률분포를 만들어 내는 것이 소프트 맥스의 역할이다.
출력층의 각 뉴런은 모든 입력 신호에서 영향을 받는다.

In [2]:
import numpy as np

a = np.array([0.3, 2.9, 4.0])

exp_a = np.exp(a) # 지수 함수
print(exp_a)

sum_exp_a = np.sum(exp_a) # 지수 함수의 합
print(sum_exp_a)

y = exp_a / sum_exp_a
print(y)

[ 1.34985881 18.17414537 54.59815003]
74.1221542101633
[0.01821127 0.24519181 0.73659691]


In [ ]:
def softmax(a):
    exp_a = np.exp(a)
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a
    
    return y

#### 3.5.2 소프트맥스 함수 구현 시 주의점

다만 위의 코드를 컴퓨터에서 계산할때는 오버플로가 발생한다.
지수함수는 매우 큰 값을 내뱉기 때문에 큰 값들끼리 나눗셈을 하면 수치가 불안정해진다.
컴퓨터는 수를 4바이트나 8바이트와 같은 유한한 크기의 데이터로 다루기 때문이다.

소프트맥스 함수를 개선해보자.
1) C라는 임의의 정수를 분자와 분모 양쪽에 곱한다.
2) C를 지수함수 exp() 안으로 옮겨 logC로 만든다.
3) logC를 새로운 기호 C'로 바꾼다.

핵심은 소프트맥스의 지수함수 계산시 어떤 정수를 더하거나 빼도 결과는 바뀌지 않는다는 것이다.
보통은 오버플로를 막을 목적으로 입력 신호 중 최댓값을 이용한다.

In [6]:
a = np.array([1010, 1000, 990])

print(np.exp(a) / np.sum(np.exp(a)))

c = np.max(a)
print(a - c)

print(np.exp(a - c) / np.sum(np.exp(a - c)))

[nan nan nan]
[  0 -10 -20]
[9.99954600e-01 4.53978686e-05 2.06106005e-09]


C:\Users\parkk\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until


In [9]:
def softmax(a):
    c = np.max(a)
    exp_a = np.exp(a-c)
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a
    
    return y

#### 3.5.3 소프트맥스 함수의 특징

소프트맥스의 출력은 0 에서 1.0 사이의 실수이며, 그 출력의 총합은 1이된다.
이러한 성질 때문에 소프트맥스 함수의 출력을 확률로 해석할 수 있다.

In [10]:
a = np.array([0.3, 2.9, 4.0])
y = softmax(a)
print(y)
print(np.sum(y))

[0.01821127 0.24519181 0.73659691]
1.0


y[0]의 확률은 0.018 (1.8%), y[1]의 확률은 0.245 (24.5%), y[2]의 확률은 0.737 (73.7%)로 해석할 수 있다.
이는 74%의 확률로 2번째 클래스, 25%의 확률로 1번째 클래스, 1%의 확률로 0번째 클래스다 라는 확률적인 결론을 낼 수 있다.

주의점은 소프트맥스 함수를 적용해도 각 원소의 대소 관계는 변하지 않는다는 것이다.
이는 지수 함수 y = exp(x)가 단조 증가 함수이기 때문이다.

신경망을 이용한 분류에서는 가장 큰 출력을 내는 뉴런에 해당하는 클래스로만 인식한다. 소프트맥스를 적용해도 위치는 달라지지 않는다. 분류당시에는 지원 낭비를 주리고자 추론 단계에서 빼기도 한다.

#### 3.5.4 출력층의 뉴런 수 정하기

풀려는 문제에 맞게 출력층의 뉴런 수를 잘 정해야 한다.
보통은 분류하고 싶은 클래스 수로 설정하는것이 일반적이다.